# Analysis of history term
In this notebook the history term calculated with the script provided in "script_history_term.py" is analysed.  
author: Meike Bos  
creation date: 26/08/2025

In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr
from datetime import datetime, timedelta
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import Divider, Size
from scipy.interpolate import CubicSpline 
plt.style.use('../python_style_Meike.mplstyle')
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")


Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.2, 0.9, 5))
from analysis_functions import make_PDF, make_lognormal_PDF, mean_angle
from particle_characteristics_functions import factor_drag_white1991, Re_particle, dynamic_viscosity_Sharqawy, stokes_relaxation_time, Oseen_time


## angle between Fdrag and history force   
I calculate the angleb etween the history force and drag force to show how much they work in the same direction or not

## NEW

In [ ]:
def prefactor_history(B,nu,d):
    return 9 * np.sqrt(nu) / (np.sqrt(np.pi) * B * d)

def prefactor_drag(B,nu,d,Rep):
    crep = factor_drag_white1991(Rep)
    return 18 * nu * crep / (B * d*d)

def calculate_magnitude(x,y):
    return np.sqrt(x**2+y**2)

In [ ]:

# density water
tau = 3196
av_temp_NWES = 11.983276
av_salinity_NWES = 34.70449
rho_water = 1027 # kg/m3 
dynamic_viscosity_water = dynamic_viscosity_Sharqawy(av_temp_NWES,av_salinity_NWES/1000)# 1.e-3# 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water
diameter = 0.25 # m
B=0.68

In [ ]:
# read in data
starttime = datetime(2023,9,1,0,0,0)
runtime = timedelta(days =2)
endtime = starttime + runtime

loc = 'NWES'
land_handling = 'anti_beaching'
Replist= [0,10,100,450,1000]
chunck_time = 100
nparticles = 52511
coriolis = True
gradient = True

base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'
basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                 'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefiles={
           'inertial_Rep_constant':basefile_Rep_constant,
        #    'inertial_SM_Rep_constant':basefile_Rep_constant, 
        #    'inertial_SM_drag_Rep':basefile_Rep_drag,
           'inertial_drag_Rep':basefile_Rep_drag}

basefile_history_mei_Rep_constant = base_directory + '{particle_type}/history_term_Mei_Rep{Rep:04d}_twindow{twindow:06d}_tresample{tresample:04d}.netcdf'
basefile_history_mei_Rep_drag = base_directory + '{particle_type}/history_term_Mei_twindow{twindow:06d}_tresample{tresample:04d}.netcdf'

basefiles_mei={
           'inertial_Rep_constant':basefile_history_mei_Rep_constant,
        #    'inertial_SM_Rep_constant':basefile_Rep_constant, 
        #    'inertial_SM_drag_Rep':basefile_Rep_drag,
           'inertial_drag_Rep':basefile_history_mei_Rep_drag}

particle_types = ['inertial_drag_Rep','inertial_Rep_constant']#,'inertial_SM_Rep_constant']#,'inertial_Rep_constant'] # 
names = {'inertial_Rep_constant':'MR',
         'inertial_SM_Rep_constant':'SM MR', 
         'inertial_SM_drag_Rep':'SM MR flexible Re$_p$',
         'inertial_drag_Rep':'MR flexible Re$_p$'}
Rep_dict = {'inertial_Rep_constant':Replist,
         'inertial_SM_Rep_constant':Replist,
         'inertial_SM_drag_Rep':[None],
         'inertial_drag_Rep':[None]}


In [ ]:
# import data
# create data directory
data = {}
for pt in particle_types:
    data[pt]={}


for pt in particle_types:
    for Rep in Rep_dict[pt]:
        file = basefiles[pt].format(loc=loc,
                                   y_s=starttime.year,
                                   m_s=starttime.month,
                                   d_s=starttime.day,
                                   y_e=endtime.year,
                                   m_e=endtime.month,
                                   d_e=endtime.day,
                                   B = int(B * 1000), 
                                   tau = int(tau ),
                                   land_handling = land_handling, 
                                   coriolis = coriolis,
                                   gradient = gradient,
                                   particle_type = pt,
                                   Rep = Rep)
        ds = xr.open_dataset(file,
                             engine='zarr',
                             chunks={'trajectory':nparticles, 'obs':chunck_time},
                             drop_variables=['B','tau','z'],
                             decode_times=False) 
        print(file)

        
        
        data[pt][Rep]= ds 

In [ ]:
legend = []
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                axes_locator=divider.new_locator(nx=1, ny=1))
for Rep,color in zip(Replist[0:],Magmalist[0:]):
    Uslip = np.sqrt(data['inertial_Rep_constant'][Rep].uslip**2 + data['inertial_Rep_constant'][Rep].vslip**2)
    toseen_values =Oseen_time(Uslip.isel(obs=slice(1,None)),kinematic_viscosity_water)
    toseen_values =toseen_values.values.flatten()
    toseen_values = toseen_values[~np.isnan(toseen_values)]
    bins, pdfs = make_lognormal_PDF(toseen_values,nbins=100,norm=True,vmin=5e-6,vmax=5e3)
    legend.append(f'c({Rep})={factor_drag_white1991(Rep):.0f}')
    ax.plot(bins[1:],pdfs[1:]* (bins[1:]-bins[:-1]),'-.',color=color)
   
Uslip = np.sqrt(data['inertial_drag_Rep'][None].uslip**2 + data['inertial_drag_Rep'][None].vslip**2)
toseen_values =Oseen_time(Uslip.isel(obs=slice(1,None)),kinematic_viscosity_water)
toseen_values =toseen_values.values.flatten()
toseen_values = toseen_values[~np.isnan(toseen_values)]
bins, pdfs = make_lognormal_PDF(toseen_values,nbins=100,norm=True,vmin=5e-6,vmax=5e3)
legend.append('c$_p$(t)')
ax.plot(bins[1:],pdfs[1:]* (bins[1:]-bins[:-1]),'--',color='cornflowerblue')
ax.set_xscale('log')
ax.legend(legend)
ax.set_ylabel('PDF')
ax.set_xlabel('Oseen time [s]')
# fig.tight_layout()
fig.savefig('../figures/history_force/PDF_oseen_time.pdf')

In [ ]:
# toseen={}

In [ ]:
# calculate X percentile values
toseen={}
percentiles=[50,95,99]
for X in percentiles:
    toseen[X]={}
    fig = plt.figure()
    h = [Size.Fixed(1.0), Size.Fixed(8)]
    v = [Size.Fixed(0.7), Size.Fixed(6)]
    divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
    ax = fig.add_axes(divider.get_position(),
                    axes_locator=divider.new_locator(nx=1, ny=1))

    
    i=1
    five_percent_list_to = []
    for Rep,color in zip(Replist,Magmalist): 
        Uslip = np.sqrt(data['inertial_Rep_constant'][Rep].uslip**2 + data['inertial_Rep_constant'][Rep].vslip**2).isel(obs=slice(1,None))
        uslip =Uslip.values.flatten()
        
        to=Oseen_time(np.nanpercentile(uslip,100-X),kinematic_viscosity_water)
        toseen[X][Rep]=to
        five_percent_list_to.append(to)
        ax.plot(i,to,'o',color=color)
        i+=1
    Rep = None
    Uslip = np.sqrt(data['inertial_drag_Rep'][None].uslip**2 + data['inertial_drag_Rep'][None].vslip**2).isel(obs=slice(1,None))
    uslip =Uslip.values.flatten()

    to=Oseen_time(np.nanpercentile(uslip,100-X),kinematic_viscosity_water)
    five_percent_list_to.append(to)
    toseen[X][Rep]=to
    ax.plot(i,to,'s',color='cornflowerblue')

    ax.set_xticks([1,2,3,4,5,6])
    ax.set_xticklabels(['c(0)','c(10)','c(100)','c(450)','c(1000)','c$_p$(t)'])
    ax.set_yscale('log')
    # ax.set_xlabel('Re$_p^{\\mathrm{In}}$')
    ax.set_ylabel(f'{X}-percentile Oseen time [s]')
    # fig.tight_layout()
    fig.savefig(f'../figures/history_force/{X:03d}_percentile_oseen_time.pdf')

    # print(X)
    # print(values)

In [ ]:
def resample_time(ds, n_resample):
    # Convert obs to numpy array
    obs = ds['obs']
    obs_vals = obs.values
    # Create new obs_resampled n_resample x the original points
    obs_resampled = np.linspace(obs_vals.min(), obs_vals.max(), num=(len(obs_vals) - 1) * n_resample + 1)
    time_resampled = ds['time'].interp(obs=('obs_resampled', obs_resampled))
    ds = ds.assign_coords(obs_resampled=obs_resampled)
    ds['time_resampled'] = time_resampled
    return ds 


def cs_resample_and_derivative(v, t, tresample):
    """
    function that takes an xarray dataarray with time coordinates and returns it resampled function and derivative 
    """
    mask = ~np.isnan(v)
    cs = CubicSpline(t[mask],v[mask])
    data_resampled = cs(tresample)
    data_derivative_resampled = cs.derivative()(tresample)
    return data_resampled, data_derivative_resampled

def calculate_basset_force(u,v,dudt,dvdt,lat,toseen,d, nu, B):
    omega_earth =  7.2921e-5
    prefactor =  9 * np.sqrt(nu)/(np.sqrt(np.pi) * B * d )
    basset_time = 2 * np.sqrt(toseen)
    f_rot = 2 * omega_earth * np.sin(np.pi*lat/180)
    gut = dudt-v*f_rot 
    gvt = dvdt+u*f_rot
    force_x = prefactor * basset_time * gut
    force_y = prefactor * basset_time * gvt
    return force_x, force_y


def calculate_drag_force(u,v, Rep, d, nu, B):
    prefactor =  18 * factor_drag_white1991(Rep) * nu / (B * d * d)
    return prefactor*u, prefactor*v

def calculate_toseen(u,v,nu):
    uslip = np.sqrt(u**2 + v**2)
    return nu/(uslip**2)

def calculate_coriolis_1(u,v,lat,B):
    omega_earth =  7.2921e-5
    f_rot = 2 * omega_earth * np.sin(np.pi*lat/180)
    return -f_rot*v/(2*B),  f_rot*u/(2*B)



In [ ]:
# now same calculation but for all particles (using apply ufunc)
def resample_time(ds, n_resample):
    # Convert obs to numpy array
    obs = ds['obs']
    obs_vals = obs.values
    # Create new obs_resampled n_resample x the original points
    obs_resampled = np.linspace(obs_vals.min(), obs_vals.max(), num=n_resample+1)
    time_resampled = ds['time'].interp(obs=('obs_resampled', obs_resampled))
    ds = ds.assign_coords(obs_resampled=obs_resampled)
    ds['time_resampled'] = time_resampled
    return ds 

def cs_resample_and_derivative(v,t, tresample):
    """
    function that takes an xarray dataarray with time coordinates and returns it resampled function and derivative 
    """
    mask = ~np.isnan(v)
    if(np.sum(mask)<2):
        nan_array = np.full_like(tresample, np.nan, dtype=float)
        return nan_array, nan_array
    else:
        cs = CubicSpline(t[mask],v[mask])
        data_resampled = cs(tresample)
        data_derivative_resampled = cs.derivative()(tresample)
        return data_resampled, data_derivative_resampled

def cs_resample(v, t, tresample):
    """
    function that takes an xarray dataarray with time coordinates and returns it resampled function
    """
    mask = ~np.isnan(v)
    if(np.sum(mask)<2):
        nan_array = np.full_like(tresample, np.nan, dtype=float)
        return nan_array, nan_array
    else:
        cs = CubicSpline(t[mask],v[mask])
        data_resampled = cs(tresample)
        return data_resampled

def calculate_basset_force(u,v,dudt,dvdt,lat,d, nu, B):
    omega_earth =  7.2921e-5
    tau=stokes_relaxation_time(d,nu,B)
    prefactor =  3 / np.sqrt(np.pi * (1+2 * B)* tau)
    uslip = np.sqrt(u**2 + v**2)
    toseen= nu/(uslip**2)
    basset_time = 2 * np.sqrt(toseen)
    f_rot = 2 * omega_earth * np.sin(np.pi*lat/180)
    gut = dudt-v*f_rot 
    gvt = dvdt+u*f_rot
    force_x = prefactor * basset_time * gut
    force_y = prefactor * basset_time * gvt
    return force_x, force_y


def calculate_basset_force_oseen_fixed(u,v,dudt,dvdt,lat,toseen,d, nu, B):
    omega_earth =  7.2921e-5
    tau=stokes_relaxation_time(d,nu,B)
    prefactor =  3 / np.sqrt(np.pi * (1+2 * B)* tau)#9 * np.sqrt(nu)/(np.sqrt(np.pi) * B * d )
    basset_time = 2 * np.sqrt(toseen)
    f_rot = 2 * omega_earth * np.sin(np.pi*lat/180)
    gut = dudt-v*f_rot 
    gvt = dvdt+u*f_rot
    force_x = prefactor * basset_time * gut
    force_y = prefactor * basset_time * gvt
    return force_x, force_y

def calculate_drag_force(u,v, Rep, d, nu, B):
    prefactor =  18 * factor_drag_white1991(Rep) * nu / (B * d * d)
    return prefactor*u, prefactor*v

def calculate_drag_force_flexible_Rep(u,v, d, nu, B):
    Rep = np.sqrt(u**2 + v**2) * d / nu
    prefactor =  18 * factor_drag_white1991(Rep) * nu / (B * d * d)
    return prefactor*u, prefactor*v

def calculate_toseen(u,v,nu):
    uslip = np.sqrt(u**2 + v**2)
    return nu/(uslip**2)




In [ ]:


basset_mag_dict={}
drag_mag_dict={}
pt='inertial_Rep_constant'
X=95
basset_mag_dict_toseen = {}
for Rep, color in zip(Replist,Magmalist): 
    print(Rep)
    nmax = 47+1#-1
    ds= data[pt][Rep].isel( obs=slice(1,nmax,1)) # trajectory=slice(0,10),
    legend.append('Re$_p^{\\mathrm{In}}=$'+f'{Rep}')
    tmax = ds.time[0].max(skipna=True).values # maximum timestep in seconds
    dt_resample=3600
    n_resample = int(tmax/dt_resample)

   
    ds = resample_time(ds, n_resample).load()
    da_uslip_cs, da_uslip_cs_derivative = xr.apply_ufunc(cs_resample_and_derivative,
                                                        ds.uslip, ds.time.values,ds.time_resampled.values, 
                                                        input_core_dims =[["obs"],["obs"],["obs_resampled"]],
                                                        output_core_dims=[["obs_resampled"],["obs_resampled"]],
                                                        vectorize=True,join="override")
    
   
    da_vslip_cs, da_vslip_cs_derivative = xr.apply_ufunc(cs_resample_and_derivative,
                                                        ds.vslip,ds.time.values,ds.time_resampled.values, 
                                                        input_core_dims =[["obs"],["obs"],["obs_resampled"]],
                                                        output_core_dims=[["obs_resampled"],["obs_resampled"]],
                                                        vectorize=True,join="override")
    da_lat_resampled = xr.apply_ufunc(cs_resample,
                                                        ds.lat,ds.time.values,ds.time_resampled.values, 
                                                        input_core_dims =[["obs"],["obs"],["obs_resampled"]],
                                                        output_core_dims=[["obs_resampled"]],
                                                        vectorize=True,join="override")
    ds = ds.assign({'uslip_resampled':da_uslip_cs,
                    'vslip_resampled':da_vslip_cs,
                    'der_uslip_resampled':da_uslip_cs_derivative,
                    'der_vslip_resampled':da_vslip_cs_derivative,
                    'lat_resampled':da_lat_resampled})
    # calculate basset force
    print('start calculating basset force')
    basset_x_toseen, basset_y_toseen = xr.apply_ufunc(calculate_basset_force_oseen_fixed,ds.uslip_resampled,ds.vslip_resampled,ds.der_uslip_resampled, ds.der_vslip_resampled,ds.lat_resampled,toseen[X][Rep],diameter,kinematic_viscosity_water,B,input_core_dims =[["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],[],[],[],[]],output_core_dims=[["obs_resampled"],["obs_resampled"]], vectorize=True)
    basset_x, basset_y = xr.apply_ufunc(calculate_basset_force,ds.uslip_resampled,ds.vslip_resampled,ds.der_uslip_resampled, ds.der_vslip_resampled,ds.lat_resampled,diameter,kinematic_viscosity_water,B,input_core_dims =[["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],[],[],[]],output_core_dims=[["obs_resampled"],["obs_resampled"]], vectorize=True)

    # calculate drag force components
    print('start calculating drag force')
    drag_x, drag_y = xr.apply_ufunc(calculate_drag_force,ds.uslip_resampled,ds.vslip_resampled,Rep,diameter,kinematic_viscosity_water,B,input_core_dims=[["obs_resampled"],["obs_resampled"],[],[],[],[]],output_core_dims=[["obs_resampled"],["obs_resampled"]], vectorize=True)

    basset_mag = np.sqrt(basset_x**2 + basset_y**2)
    basset_mag_toseen = np.sqrt(basset_x_toseen**2 + basset_y_toseen**2)
    drag_mag = np.sqrt(drag_x**2 + drag_y**2)
    #caluclate ratio basset and drag force
    fraction = basset_mag/drag_mag
    basset_mag_dict[Rep]=basset_mag
    basset_mag_dict_toseen[Rep]=basset_mag_toseen
    drag_mag_dict[Rep]=drag_mag


#also flexible Rep
pt='inertial_drag_Rep'
Rep = None 
nmax = 47+1#-1
ds= data[pt][Rep].isel( obs=slice(1,nmax,1)) # trajectory=slice(0,10),
tmax = ds.time[0].max(skipna=True).values # maximum timestep in seconds
dt_resample=3600
n_resample = int(tmax/dt_resample)

   
ds = resample_time(ds, n_resample).load()
da_uslip_cs, da_uslip_cs_derivative = xr.apply_ufunc(cs_resample_and_derivative,
                                                    ds.uslip, ds.time.values,ds.time_resampled.values, 
                                                    input_core_dims =[["obs"],["obs"],["obs_resampled"]],
                                                    output_core_dims=[["obs_resampled"],["obs_resampled"]],
                                                    vectorize=True,join="override")


da_vslip_cs, da_vslip_cs_derivative = xr.apply_ufunc(cs_resample_and_derivative,
                                                    ds.vslip,ds.time.values,ds.time_resampled.values, 
                                                    input_core_dims =[["obs"],["obs"],["obs_resampled"]],
                                                    output_core_dims=[["obs_resampled"],["obs_resampled"]],
                                                    vectorize=True,join="override")
da_lat_resampled = xr.apply_ufunc(cs_resample,
                                                    ds.lat,ds.time.values,ds.time_resampled.values, 
                                                    input_core_dims =[["obs"],["obs"],["obs_resampled"]],
                                                    output_core_dims=[["obs_resampled"]],
                                                    vectorize=True,join="override")
ds = ds.assign({'uslip_resampled':da_uslip_cs,
                'vslip_resampled':da_vslip_cs,
                'der_uslip_resampled':da_uslip_cs_derivative,
                'der_vslip_resampled':da_vslip_cs_derivative,
                'lat_resampled':da_lat_resampled})
# calculate basset force
print('start calculating basset force')
basset_x_toseen, basset_y_toseen = xr.apply_ufunc(calculate_basset_force_oseen_fixed,ds.uslip_resampled,ds.vslip_resampled,ds.der_uslip_resampled, ds.der_vslip_resampled,ds.lat_resampled,toseen[X][Rep],diameter,kinematic_viscosity_water,B,input_core_dims =[["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],[],[],[],[]],output_core_dims=[["obs_resampled"],["obs_resampled"]], vectorize=True)
basset_x, basset_y = xr.apply_ufunc(calculate_basset_force,ds.uslip_resampled,ds.vslip_resampled,ds.der_uslip_resampled, ds.der_vslip_resampled,ds.lat_resampled,diameter,kinematic_viscosity_water,B,input_core_dims =[["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],[],[],[]],output_core_dims=[["obs_resampled"],["obs_resampled"]], vectorize=True)

# calculate drag force components
print('start calculating drag force')
drag_x, drag_y = xr.apply_ufunc(calculate_drag_force_flexible_Rep,ds.uslip_resampled,ds.vslip_resampled,diameter,kinematic_viscosity_water,B,input_core_dims=[["obs_resampled"],["obs_resampled"],[],[],[]],output_core_dims=[["obs_resampled"],["obs_resampled"]], vectorize=True)

basset_mag = np.sqrt(basset_x**2 + basset_y**2)
basset_mag_toseen = np.sqrt(basset_x_toseen**2 + basset_y_toseen**2)
drag_mag = np.sqrt(drag_x**2 + drag_y**2)
#caluclate ratio basset and drag force
fraction = basset_mag/drag_mag
basset_mag_dict[Rep]=basset_mag
basset_mag_dict_toseen[Rep]=basset_mag_toseen
drag_mag_dict[Rep]=drag_mag

In [ ]:
fig, ax = plt.subplots()
alphalist=[0.1,0.3,0.5,0.7,0.9]
Replist=[0,10,100,450,1000]
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()
id=25
pt = 'inertial_Rep_constant'

for Rep, alpha in zip(Replist,alphalist):
    # if (Rep ==450): 
    #     continue
    
    ds = data[pt][Rep]
    file_mei = basefiles_mei[pt].format(Rep=Rep,
                                        particle_type=pt,
                                        twindow=720000,
                                        tresample=100)
    print(file_mei)
    ds_mei = xr.open_dataset(file_mei)

    # magnitude of forces


    flift_mag = calculate_magnitude(ds.flift_x,ds.flift_y)
    fcor_mag = calculate_magnitude(ds.fcor_x,ds.fcor_y)
    fgradient_mag =  calculate_magnitude(ds.fgradient_x,ds.fgradient_y)
    fdrag_mag = factor_drag_white1991(Rep) / tau * calculate_magnitude(ds.uslip,ds.vslip)
    prefactor =  3 / np.sqrt(np.pi * (1+2 * B)* tau)
    mei_mag =prefactor * calculate_magnitude(ds_mei.History_Mei_x,ds_mei.History_Mei_y)

    flift_mag_list = (flift_mag.values).flatten()
    flift_mag_list = flift_mag_list[~np.isnan(flift_mag_list)]
    bins_l, pdf_l = make_lognormal_PDF(flift_mag_list,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e-3)
    fcor_mag_list = fcor_mag.values.flatten()
    fcor_mag_list = fcor_mag_list[~np.isnan(fcor_mag_list)]
    bins_c, pdf_c = make_lognormal_PDF(fcor_mag_list,nbins=100,norm=True ,vmin =  1e-9,vmax  =  1e-3)
    fgradient_mag_list = fgradient_mag.values.flatten()
    fgradient_mag_list =fgradient_mag_list[~np.isnan(fgradient_mag_list)]
    bins_g, pdf_g = make_lognormal_PDF(fgradient_mag_list,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e-3)
    fdrag_mag_list = fdrag_mag.values.flatten()
    fdrag_mag_list =fdrag_mag_list[~np.isnan(fdrag_mag_list)]
    bins_d, pdf_d = make_lognormal_PDF(fdrag_mag_list,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e-3)
    ax.plot(bins_c[1:], pdf_c[1:]*(bins_c[1:]-bins_c[:-1]), '--', color = 'cornflowerblue',alpha=alpha)
    ax.plot(bins_g[1:], pdf_g[1:]*(bins_g[1:]-bins_g[:-1]), '--', color = 'orange',alpha=alpha)
    ax.plot(bins_l[1:], pdf_l[1:]*(bins_l[1:]-bins_l[:-1]), '--', color = 'navy',alpha=alpha)
    ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '--', color = 'firebrick',alpha=alpha)
    basset_mag=basset_mag_dict[Rep]
    basset_mag =(basset_mag.values).flatten()
    basset_mag = basset_mag[~np.isnan(basset_mag)]
    bins_b, pdf_b = make_lognormal_PDF(basset_mag,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e-3)
    ax.plot(bins_b[1:], pdf_b[1:]*(bins_b[1:]-bins_b[:-1]), '--', color = 'purple',alpha=alpha)
    mei_mag_list = (mei_mag.values).flatten()
    mei_mag_list = mei_mag_list[~np.isnan(mei_mag_list)]
    bins_m, pdf_m = make_lognormal_PDF(mei_mag_list,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e-3)
    ax.plot(bins_m[1:], pdf_m[1:]*(bins_m[1:]-bins_m[:-1]), '--', color = 'green',alpha=alpha)

    
    ax2.plot(ds.time.isel(trajectory=id)/3600,ds.flift_x.isel(trajectory=id),'--o',color='navy',alpha=alpha)
    ax2.plot(ds.time.isel(trajectory=id)/3600,ds.fcor_x.isel(trajectory=id),'--o',color='cornflowerblue',alpha=alpha)
    ax2.plot(ds.time.isel(trajectory=id)/3600,ds.fgradient_x.isel(trajectory=id),'--o',color='orange',alpha=alpha)
    ax2.plot(ds.time.isel(trajectory=id)/3600,factor_drag_white1991(Rep) / tau * ds.uslip.isel(trajectory=id),'--o',color='firebrick',alpha=alpha)
    ax2.plot(ds_mei.time/3600,prefactor*ds_mei.History_Mei_x.isel(trajectory=id),'--o',color='green',alpha=alpha)

    ax3.plot(ds.time.isel(trajectory=id)/3600,ds.flift_y.isel(trajectory=id),'--o',color='navy',alpha=alpha)
    ax3.plot(ds.time.isel(trajectory=id)/3600,ds.fcor_y.isel(trajectory=id),'--o',color='cornflowerblue',alpha=alpha)
    ax3.plot(ds.time.isel(trajectory=id)/3600,ds.fgradient_y.isel(trajectory=id),'--o',color='orange',alpha=alpha)
    ax3.plot(ds.time.isel(trajectory=id)/3600,factor_drag_white1991(Rep) / tau * ds.vslip.isel(trajectory=id),'--o',color='firebrick',alpha=alpha)
    ax3.plot(ds_mei.time/3600,prefactor*ds_mei.History_Mei_y.isel(trajectory=id),'--o',color='green',alpha=alpha)

pt = 'inertial_drag_Rep'
Rep=None
alpha=1
ds = data[pt][Rep]
file_mei = basefiles_mei[pt].format(Rep=Rep,
                                    particle_type=pt,
                                    twindow=720000,
                                    tresample=100)
print(file_mei)
ds_mei = xr.open_dataset(file_mei)

# magnitude of forces

flift_mag = calculate_magnitude(ds.flift_x,ds.flift_y)
fcor_mag = calculate_magnitude(ds.fcor_x,ds.fcor_y)
fgradient_mag =  calculate_magnitude(ds.fgradient_x,ds.fgradient_y)
mag_uslip = calculate_magnitude(ds.uslip,ds.vslip)
Replist = factor_drag_white1991(Re_particle(mag_uslip,diameter=diameter,kinematic_viscosity=kinematic_viscosity_water))
fdrag_mag = Replist / tau * mag_uslip
prefactor =  3 / np.sqrt(np.pi * (1+2 * B)* tau)
mei_mag =prefactor * calculate_magnitude(ds_mei.History_Mei_x,ds_mei.History_Mei_y)

flift_mag_list = (flift_mag.values).flatten()
flift_mag_list = flift_mag_list[~np.isnan(flift_mag_list)]
bins_l, pdf_l = make_lognormal_PDF(flift_mag_list,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e-3)
fcor_mag_list = fcor_mag.values.flatten()
fcor_mag_list = fcor_mag_list[~np.isnan(fcor_mag_list)]
bins_c, pdf_c = make_lognormal_PDF(fcor_mag_list,nbins=100,norm=True ,vmin =  1e-9,vmax  =  1e-3)
fgradient_mag_list = fgradient_mag.values.flatten()
fgradient_mag_list =fgradient_mag_list[~np.isnan(fgradient_mag_list)]
bins_g, pdf_g = make_lognormal_PDF(fgradient_mag_list,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e-3)
fdrag_mag_list = fdrag_mag.values.flatten()
fdrag_mag_list =fdrag_mag_list[~np.isnan(fdrag_mag_list)]
bins_d, pdf_d = make_lognormal_PDF(fdrag_mag_list,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e-3)
ax.plot(bins_c[1:], pdf_c[1:]*(bins_c[1:]-bins_c[:-1]), '-', color = 'cornflowerblue',alpha=alpha)
ax.plot(bins_g[1:], pdf_g[1:]*(bins_g[1:]-bins_g[:-1]), '-', color = 'orange',alpha=alpha)
ax.plot(bins_l[1:], pdf_l[1:]*(bins_l[1:]-bins_l[:-1]), '-', color = 'navy',alpha=alpha)
ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = 'firebrick',alpha=alpha)
basset_mag=basset_mag_dict[Rep]
basset_mag =(basset_mag.values).flatten()
basset_mag = basset_mag[~np.isnan(basset_mag)]
bins_b, pdf_b = make_lognormal_PDF(basset_mag,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e-3)
ax.plot(bins_b[1:], pdf_b[1:]*(bins_b[1:]-bins_b[:-1]), '-', color = 'purple',alpha=alpha)
mei_mag = (mei_mag.values).flatten()
mei_mag = mei_mag[~np.isnan(mei_mag)]
bins_m, pdf_m = make_lognormal_PDF(mei_mag,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e-3)
ax.plot(bins_m[1:], pdf_m[1:]*(bins_m[1:]-bins_m[:-1]), '-', color = 'green',alpha=alpha)


ax2.plot(ds.time.isel(trajectory=id)/3600,ds.flift_x.isel(trajectory=id),'--d',color='navy',alpha=alpha)
ax2.plot(ds.time.isel(trajectory=id)/3600,ds.fcor_x.isel(trajectory=id),'--d',color='cornflowerblue',alpha=alpha)
ax2.plot(ds.time.isel(trajectory=id)/3600,ds.fgradient_x.isel(trajectory=id),'--d',color='orange',alpha=alpha)
ax2.plot(ds.time.isel(trajectory=id)/3600,(Replist*ds.uslip).isel(trajectory=id)/tau,'--d',color='firebrick',alpha=alpha)
ax3.plot(ds_mei.time/3600,prefactor*ds_mei.History_Mei_x.isel(trajectory=id),'--d',color='green',alpha=alpha)

ax3.plot(ds.time.isel(trajectory=id)/3600,ds.flift_y.isel(trajectory=id),'--d',color='navy',alpha=alpha)
ax3.plot(ds.time.isel(trajectory=id)/3600,ds.fcor_y.isel(trajectory=id),'--d',color='cornflowerblue',alpha=alpha)
ax3.plot(ds.time.isel(trajectory=id)/3600,ds.fgradient_y.isel(trajectory=id),'--d',color='orange',alpha=alpha)
ax3.plot(ds.time.isel(trajectory=id)/3600,(Replist*ds.vslip).isel(trajectory=id)/tau,'--d',color='firebrick',alpha=alpha)
ax3.plot(ds_mei.time/3600,prefactor*ds_mei.History_Mei_y.isel(trajectory=id),'--d',color='green',alpha=alpha)



ax.set_xscale('log')

In [ ]:
# plot per rep value
for Rep in [0,10,100,450,1000]:
    fig = plt.figure()
    h = [Size.Fixed(1.0), Size.Fixed(8)]
    v = [Size.Fixed(0.7), Size.Fixed(6)]
    divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
    ax = fig.add_axes(divider.get_position(),
                    axes_locator=divider.new_locator(nx=1, ny=1))
    
    print(Rep)
    pt = 'inertial_Rep_constant'
    ds = data[pt][Rep]

    file_mei = basefiles_mei[pt].format(Rep=Rep,
                    particle_type=pt,
                    twindow=720000,
                    tresample=100)
    ds_mei = xr.open_dataset(file_mei)

    # magnitude of forces


    flift_mag = calculate_magnitude(ds.flift_x,ds.flift_y)
    fcor_mag = calculate_magnitude(ds.fcor_x,ds.fcor_y)
    fgradient_mag =  calculate_magnitude(ds.fgradient_x,ds.fgradient_y)
    fdrag_mag = factor_drag_white1991(Rep) / tau * calculate_magnitude(ds.uslip,ds.vslip)
    prefactor =  3 / np.sqrt(np.pi * (1+2 * B)* tau)
    mei_mag =prefactor * calculate_magnitude(ds_mei.History_Mei_x,ds_mei.History_Mei_y)

    flift_mag_list = (flift_mag.values).flatten()
    flift_mag_list = flift_mag_list[~np.isnan(flift_mag_list)]
    bins_l, pdf_l = make_lognormal_PDF(flift_mag_list,nbins=100,norm=True ,vmin =   1e-10,vmax  = 1e-3)
    fcor_mag_list = fcor_mag.values.flatten()
    fcor_mag_list = fcor_mag_list[~np.isnan(fcor_mag_list)]
    bins_c, pdf_c = make_lognormal_PDF(fcor_mag_list,nbins=100,norm=True ,vmin =   1e-10,vmax  =  1e-3)
    fgradient_mag_list = fgradient_mag.values.flatten()
    fgradient_mag_list =fgradient_mag_list[~np.isnan(fgradient_mag_list)]
    bins_g, pdf_g = make_lognormal_PDF(fgradient_mag_list,nbins=100,norm=True ,vmin =  1e-10,vmax  = 1e-3)
    fdrag_mag_list = fdrag_mag.values.flatten()
    fdrag_mag_list =fdrag_mag_list[~np.isnan(fdrag_mag_list)]
    bins_d, pdf_d = make_lognormal_PDF(fdrag_mag_list,nbins=100,norm=True ,vmin =   1e-10,vmax  = 1e-3)
    ax.plot(bins_g[1:], pdf_g[1:]*(bins_g[1:]-bins_g[:-1]), '-', color = 'orange',alpha=alpha)
    ax.plot(bins_c[1:], pdf_c[1:]*(bins_c[1:]-bins_c[:-1]), '-', color = 'cornflowerblue',alpha=alpha)
    ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '--', color = 'firebrick',alpha=alpha)
    ax.plot(bins_l[1:], pdf_l[1:]*(bins_l[1:]-bins_l[:-1]), ':', color = 'navy',alpha=alpha)

    basset_mag=basset_mag_dict[Rep]
    basset_mag =(basset_mag.values).flatten()
    basset_mag = basset_mag[~np.isnan(basset_mag)]
    bins_b, pdf_b = make_lognormal_PDF(basset_mag,nbins=100,norm=True ,vmin =   1e-10,vmax  = 1e-3)
    ax.plot(bins_b[1:], pdf_b[1:]*(bins_b[1:]-bins_b[:-1]), '-.', color = 'purple',alpha=alpha)
    mei_mag_list = (mei_mag.values).flatten()
    mei_mag_list = mei_mag_list[~np.isnan(mei_mag_list)]
    bins_m, pdf_m = make_lognormal_PDF(mei_mag_list,nbins=100,norm=True ,vmin =   1e-10,vmax  = 1e-3)
    ax.plot(bins_m[1:], pdf_m[1:]*(bins_m[1:]-bins_m[:-1]), '-.', color = 'green',alpha=alpha)
    ax.set_title(f'c({Rep})={factor_drag_white1991(Rep):.0f}',fontsize=18)
    ax.legend(['inertia fluid','Coriolis','drag','lift','Basset with Oseen time','Mei/Adrian'],fontsize=18)

    ax.set_xscale('log')
    ax.set_ylabel('PDF')
    ax.set_xlabel('acceleration [m/s$^2$]')
    fig.savefig(f'../figures/history_force/PDFS_acceleration_Rep{Rep:04d}.pdf')


In [ ]:
# plot per rep value
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1))
pt = 'inertial_drag_Rep'
Rep=None
ds = data[pt][Rep]
file_mei = basefiles_mei[pt].format(Rep=Rep,
                particle_type=pt,
                twindow=720000,
                tresample=100)
print(file_mei)
ds_mei = xr.open_dataset(file_mei)

# magnitude of forces


flift_mag = calculate_magnitude(ds.flift_x,ds.flift_y)
fcor_mag = calculate_magnitude(ds.fcor_x,ds.fcor_y)
fgradient_mag =  calculate_magnitude(ds.fgradient_x,ds.fgradient_y)
mag_uslip = calculate_magnitude(ds.uslip,ds.vslip)
Replist = factor_drag_white1991(Re_particle(mag_uslip,diameter=diameter,kinematic_viscosity=kinematic_viscosity_water))
fdrag_mag = Replist / tau * mag_uslip
prefactor =  3 / np.sqrt(np.pi * (1+2 * B)* tau)
mei_mag =prefactor * calculate_magnitude(ds_mei.History_Mei_x,ds_mei.History_Mei_y)

flift_mag_list = (flift_mag.values).flatten()
flift_mag_list = flift_mag_list[~np.isnan(flift_mag_list)]
bins_l, pdf_l = make_lognormal_PDF(flift_mag_list,nbins=100,norm=True ,vmin =  1e-10,vmax  = 1e-3)
fcor_mag_list = fcor_mag.values.flatten()
fcor_mag_list = fcor_mag_list[~np.isnan(fcor_mag_list)]
bins_c, pdf_c = make_lognormal_PDF(fcor_mag_list,nbins=100,norm=True ,vmin =   1e-10,vmax  =  1e-3)
fgradient_mag_list = fgradient_mag.values.flatten()
fgradient_mag_list =fgradient_mag_list[~np.isnan(fgradient_mag_list)]
bins_g, pdf_g = make_lognormal_PDF(fgradient_mag_list,nbins=100,norm=True ,vmin =   1e-10,vmax  = 1e-3)
fdrag_mag_list = fdrag_mag.values.flatten()
fdrag_mag_list =fdrag_mag_list[~np.isnan(fdrag_mag_list)]
bins_d, pdf_d = make_lognormal_PDF(fdrag_mag_list,nbins=100,norm=True ,vmin =   1e-10,vmax  = 1e-3)
ax.plot(bins_g[1:], pdf_g[1:]*(bins_g[1:]-bins_g[:-1]), '-', color = 'orange',alpha=alpha)
ax.plot(bins_c[1:], pdf_c[1:]*(bins_c[1:]-bins_c[:-1]), '-', color = 'cornflowerblue',alpha=alpha)
ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '--', color = 'firebrick',alpha=alpha)
ax.plot(bins_l[1:], pdf_l[1:]*(bins_l[1:]-bins_l[:-1]), ':', color = 'navy',alpha=alpha)

basset_mag=basset_mag_dict[Rep]
basset_mag =(basset_mag.values).flatten()
basset_mag = basset_mag[~np.isnan(basset_mag)]
bins_b, pdf_b = make_lognormal_PDF(basset_mag,nbins=100,norm=True ,vmin =  1e-10,vmax  = 1e-3)
ax.plot(bins_b[1:], pdf_b[1:]*(bins_b[1:]-bins_b[:-1]), '-.', color = 'purple',alpha=alpha)
mei_mag_list = (mei_mag.values).flatten()
mei_mag_list = mei_mag_list[~np.isnan(mei_mag_list)]
bins_m, pdf_m = make_lognormal_PDF(mei_mag_list,nbins=100,norm=True ,vmin =   1e-10,vmax  = 1e-3)
ax.plot(bins_m[1:], pdf_m[1:]*(bins_m[1:]-bins_m[:-1]), '-.', color = 'green',alpha=alpha)
ax.set_title('c$_p$(t)',fontsize=18)
ax.legend(['inertia fluid','Coriolis','drag','lift','Basset with Oseen time','Mei/Adrian'],fontsize=18)

ax.set_xscale('log')
ax.set_ylabel('PDF')
ax.set_xlabel('acceleration [m/s$^2$]')
fig.savefig(f'../figures/history_force/PDFS_acceleration_Rep_flexible.pdf')


In [ ]:
id = 25
fig, ax = plt.subplots()
ax.plot(ds_test.time.isel(trajectory=id)/3600,ds_test.flift_x.isel(trajectory=id),'--o',color='navy')
ax.plot(ds_test.time.isel(trajectory=id)/3600,ds_test.fcor_x.isel(trajectory=id),'--o',color='cornflowerblue')
ax.plot(ds_test.time.isel(trajectory=id)/3600,ds_test.fgradient_x.isel(trajectory=id),'--o',color='orange')
ax.plot(ds_test.time.isel(trajectory=id)/3600,factor_drag_white1991(Rep) / tau * ds_test.uslip.isel(trajectory=id),'--o',color='firebrick')
# ax.set_yscale('log')

In [ ]:
ds_test.vf.isel(trajectory=id).values



In [ ]:


Replist = [0,10,100,450,1000]
twindowlist = [1,2,8,54,179,12]
path = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'
basefile_constant = '{pt}/history_term_Mei_Rep{Rep:04d}_twindow{twindow:03d}_tresample{tresample:03d}.netcdf'
basefile_flex ='{pt}/history_term_Mei_twindow{twindow:03d}_tresample{tresample:03d}.netcdf'
particle_types = ['inertial_Rep_constant','inertial_drag_Rep']

Repvalues = { 'inertial_Rep_constant':Replist,
             'inertial_drag_Rep':[None],
             
}

basefiles = {'inertial_Rep_constant':path+basefile_constant, 
             'inertial_drag_Rep':path+basefile_flex,
}


In [ ]:
ds = xr.open_dataset(path+'inertial_Rep_constant/NWES_start2023_09_01_end2023_09_03_RK4__Rep_0100_B0680_tau3196_anti_beaching_cor_True_gradient_True.zarr')
print(ds)

In [ ]:
#settings
twindow = 20 #h
tresample = 100#s

data = {}
for pt in particle_types:
    data[pt]={}

for pt in particle_types:
    for Rep in Repvalues[pt]:
        file = basefiles[pt].format(pt=pt,
                                    Rep=Rep,
                                    twindow=twindow,
                                    tresample=tresample)
        print(file)
        ds=xr.open_dataset(file)
        data[pt][Rep]=ds

In [ ]:
# plot history force together with all other forces
def drag_force(uslip,d,nu,B,Rep=-1):
    if(Rep==-1):
        Rep = uslip * d /nu
    crep= factor_drag_white1991(Rep)
    prefactor= 18 * nu * crep / (B * d**2)
    return prefactor * uslip 

def coriolis_force(uslip, up,B,):
    omega_earth = 7.2921 * (10**-5)
    


In [ ]:
fig, ax = plt.subplots()
drag_meanlist = []
drag_sdlist = []
history_meanlist = []
history_sdlist = []
legend = []
pt = 'inertial_Rep_constant'
for Rep,color in zip(Replist, Magmalist):


    Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
    Uslip = Uslip.flatten()
    Uslip = Uslip[~np.isnan(Uslip)]
    Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
    factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Repvalue)
    factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
    history_force = factor_history * np.sqrt(data[pt][Rep].History_Mei_y**2 + data[pt][Rep].History_Mei_y**2 ).values
    history_force.flatten()
    history_force = history_force[~np.isnan(history_force)] # [mask]
    drag_force = factor_drag * Uslip
    bins_h, pdf_h = make_lognormal_PDF(history_force,nbins=100,norm=True, vmin = 1e-9,vmax  =1e0)
    bins_d, pdf_d = make_lognormal_PDF(drag_force,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e0)
    drag_meanlist.append(np.mean(drag_force))
    drag_sdlist.append(np.std(drag_force))
    history_meanlist.append(np.mean(history_force))
    history_sdlist.append(np.std(history_force))
    legend.append('Re$_p^{\\mathrm{In}}$ = '+f'{Rep}')
    
    ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = color)
    ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = color, label='_nolegend_')
    ax.set_xscale('log')


pt = 'inertial_drag_Rep'
Rep = None
Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
Uslip = Uslip.flatten()
Uslip = Uslip[~np.isnan(Uslip)]
Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Repvalue)
factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
history_force = factor_history * np.sqrt(data[pt][Rep].History_Mei_y**2 + data[pt][Rep].History_Mei_y**2 ).values
history_force.flatten()
history_force = history_force[~np.isnan(history_force)] # [mask]
drag_force = factor_drag * Uslip
bins_h, pdf_h = make_lognormal_PDF(history_force,nbins=100,norm=True, vmin = 1e-9,vmax  =1e0)
bins_d, pdf_d = make_lognormal_PDF(drag_force,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e0)
drag_meanlist.append(np.mean(drag_force))
drag_sdlist.append(np.std(drag_force))
history_meanlist.append(np.mean(history_force))
history_sdlist.append(np.std(history_force))
ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = 'cornflowerblue')
ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = 'cornflowerblue',label='_nolegend_')
legend.append('Flexible Re$_p$')

ax.legend(legend,fontsize=18,loc=1)
ax.set_ylabel('PDF')
ax.set_xlabel('Force / m$_p$')
ax.text(1e-5,0.062,'Drag',color='black',fontsize=18)
ax.text(5e-7,0.062,'History',color='black',fontsize=18)
# ax.set_xlim(1e-8,1e-2)
# ax.set_ylim(-0.001,0.07)
drag_meanlist = np.asarray(drag_meanlist)
drag_sdlist = np.asarray(drag_sdlist)
history_meanlist = np.asarray(history_meanlist)
history_sdlist = np.asarray(history_sdlist)

fig.tight_layout()
# fig.savefig('../figures/history_force/history_mei_drag_distributions.pdf')

In [ ]:
fig, ax = plt.subplots()
drag_meanlist = []
drag_sdlist = []
history_meanlist = []
history_sdlist = []
legend = []
pt = 'inertial_Rep_constant'
for Rep,color in zip(Replist, Magmalist):


    Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
    Uslip = Uslip.flatten()
    Uslip = Uslip[~np.isnan(Uslip)]
    # Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
    factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Rep)
    factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
    history_force = factor_history * np.sqrt(data[pt][Rep].History_Mei_y**2 + data[pt][Rep].History_Mei_y**2 ).values
    history_force.flatten()
    history_force = history_force[~np.isnan(history_force)] # [mask]
    drag_force = factor_drag * Uslip
    bins_h, pdf_h = make_lognormal_PDF(history_force,nbins=100,norm=True, vmin = 1e-9,vmax  =1e0)
    bins_d, pdf_d = make_lognormal_PDF(drag_force,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e0)
    drag_meanlist.append(np.mean(drag_force))
    drag_sdlist.append(np.std(drag_force))
    history_meanlist.append(np.mean(history_force))
    history_sdlist.append(np.std(history_force))
    legend.append('Re$_p^{\\mathrm{In}}$ = '+f'{Rep}')
    
    ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = color)
    ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = color, label='_nolegend_')
    ax.set_xscale('log')


pt = 'inertial_drag_Rep'
Rep = None
Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
Uslip = Uslip.flatten()
Uslip = Uslip[~np.isnan(Uslip)]
Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Repvalue)
factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
history_force = factor_history * np.sqrt(data[pt][Rep].History_Mei_y**2 + data[pt][Rep].History_Mei_y**2 ).values
history_force.flatten()
history_force = history_force[~np.isnan(history_force)] # [mask]
drag_force = factor_drag * Uslip
bins_h, pdf_h = make_lognormal_PDF(history_force,nbins=100,norm=True, vmin = 1e-9,vmax  =1e0)
bins_d, pdf_d = make_lognormal_PDF(drag_force,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e0)
drag_meanlist.append(np.mean(drag_force))
drag_sdlist.append(np.std(drag_force))
history_meanlist.append(np.mean(history_force))
history_sdlist.append(np.std(history_force))
ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = 'cornflowerblue')
ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = 'cornflowerblue',label='_nolegend_')
legend.append('Flexible Re$_p$')

ax.legend(legend,fontsize=18,loc=1)
ax.set_ylabel('PDF')
ax.set_xlabel('Force / m$_p$')
ax.text(1e-5,0.062,'Drag',color='black',fontsize=18)
ax.text(5e-7,0.062,'History',color='black',fontsize=18)
# ax.set_xlim(1e-8,1e-2)
# ax.set_ylim(-0.001,0.07)
drag_meanlist = np.asarray(drag_meanlist)
drag_sdlist = np.asarray(drag_sdlist)
history_meanlist = np.asarray(history_meanlist)
history_sdlist = np.asarray(history_sdlist)

fig.tight_layout()
# fig.savefig('../figures/history_force/history_mei_drag_distributions.pdf')

In [ ]:
fig, ax = plt.subplots()
drag_meanlist = []
drag_sdlist = []
history_meanlist = []
history_sdlist = []
legend = []
pt = 'inertial_Rep_constant'
for Rep,color in zip(Replist, Magmalist):


    Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
    Uslip = Uslip.flatten()
    # Uslip = Uslip[~np.isnan(Uslip)]
    # Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
    factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Rep)
    factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
    history_force = factor_history * np.sqrt(data[pt][Rep].History_Mei_y**2 + data[pt][Rep].History_Mei_y**2 ).values
    history_force=history_force.flatten()
    # history_force = history_force[~np.isnan(history_force)] # [mask]
    drag_force = factor_drag * Uslip
    drag_force = drag_force.flatten()
    ratio = history_force/drag_force
    ratio = ratio[~np.isnan(ratio)]
    bins_h, pdf_h = make_lognormal_PDF(ratio,nbins=100,norm=True, vmin = 1e-4,vmax  =1e2)
    # bins_d, pdf_d = make_lognormal_PDF(drag_force,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e0)
    drag_meanlist.append(np.mean(drag_force))
    drag_sdlist.append(np.std(drag_force))
    history_meanlist.append(np.mean(history_force))
    history_sdlist.append(np.std(history_force))
    legend.append('Re$_p^{\\mathrm{In}}$ = '+f'{Rep}')
    
    ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = color)
    # ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = color, label='_nolegend_')
    ax.set_xscale('log')


pt = 'inertial_drag_Rep'
Rep = None
Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
Uslip = Uslip.flatten()
# Uslip = Uslip[~np.isnan(Uslip)]
Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Repvalue)
factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
history_force = factor_history * np.sqrt(data[pt][Rep].History_Mei_y**2 + data[pt][Rep].History_Mei_y**2 ).values
history_force=history_force.flatten()
# history_force = history_force[~np.isnan(history_force)] # [mask]
drag_force = factor_drag * Uslip
darg_force = drag_force.flatten()
ratio = history_force/drag_force
ratio = ratio[~np.isnan(ratio)]
bins_h, pdf_h = make_lognormal_PDF(ratio,nbins=100,norm=True, vmin = 1e-4,vmax  =1e2)
# bins_d, pdf_d = make_lognormal_PDF(drag_force,nbins=100,norm=True ,vmin =  1e-9,vmax  = 1e0)
drag_meanlist.append(np.mean(drag_force))
drag_sdlist.append(np.std(drag_force))
history_meanlist.append(np.mean(history_force))
history_sdlist.append(np.std(history_force))
ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = 'cornflowerblue')
# ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = 'cornflowerblue',label='_nolegend_')
legend.append('Flexible Re$_p$')

ax.legend(legend,fontsize=18,loc=1)
ax.set_ylabel('PDF')
ax.set_xlabel('Force / m$_p$')

# ax.set_xlim(1e-8,1e-2)
# ax.set_ylim(-0.001,0.07)
drag_meanlist = np.asarray(drag_meanlist)
drag_sdlist = np.asarray(drag_sdlist)
history_meanlist = np.asarray(history_meanlist)
history_sdlist = np.asarray(history_sdlist)

fig.tight_layout()
# fig.savefig('../figures/history_force/history_mei_drag_distributions.pdf')

In [ ]:
fig, ax =plt.subplots()
# ax.errorbar(Replist,drag_meanlist,drag_sdlist,fmt='-o')
# ax.errorbar(Replist,history_meanlist,history_sdlist,fmt='--s')
ilist = [0,1,2,3]
for i, color in zip(ilist,Magmalist[1:]):
    # ax.plot(i,history_meanlist[i]/drag_meanlist[i],'s',color=color)
    ax.errorbar(i,drag_meanlist[i],drag_sdlist[i]/2,fmt='o',color=color)
    ax.errorbar(i,history_meanlist[i],history_sdlist[i]/2,fmt='s',color=color)
# ax.plot(4,history_meanlist[4]/drag_meanlist[4],'s',color='cornflowerblue')
i=4
ax.errorbar(i,drag_meanlist[i],drag_sdlist[i]/2,fmt='o',color='cornflowerblue')
ax.errorbar(i,history_meanlist[i],history_sdlist[i]/2,fmt='s',color='cornflowerblue')
ax.set_xticks([0,1,2,3,4])
ax.set_xticklabels([10,100,450,100,'flexible'])
ax.set_yscale('log')
# drag_meanlist

# basset with twindow from oseen time

In [ ]:
# density water
tau = 3196
av_temp_NWES = 11.983276
av_salinity_NWES = 34.70449
rho_water = 1027 # kg/m3 
dynamic_viscosity_water = dynamic_viscosity_Sharqawy(av_temp_NWES,av_salinity_NWES/1000)# 1.e-3# 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water
diameter = 0.25 # m
B=0.68

Replist = [0,10,100,450,1000]
twindowlist = [1,2,8,54,179]
path = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'
basefile_constant = '{pt}/history_term_Basset_Rep{Rep:04d}_twindow{twindow:05d}_tresample{tresample:03d}.netcdf'
basefile_flex ='{pt}/history_term_Basset_twindow{twindow:05d}_tresample{tresample:03d}.netcdf'
particle_types = ['inertial_Rep_constant','inertial_drag_Rep']

Repvalues = { 'inertial_Rep_constant':Replist,
             'inertial_drag_Rep':[None]
             
}
twindowvalues =  { 'inertial_Rep_constant':twindowlist,
             'inertial_drag_Rep':[12]
             
}

basefiles = {'inertial_Rep_constant':path+basefile_constant, 
             'inertial_drag_Rep':path+basefile_flex}

In [ ]:
#settings
# twindow = 20 #h
tresample = 1#s

data = {}
for pt in particle_types:
    data[pt]={}

for pt in particle_types:
    for Rep,twindow in zip(Repvalues[pt],twindowvalues[pt]):
        file = basefiles[pt].format(pt=pt,
                                    Rep=Rep,
                                    twindow=twindow,
                                    tresample=tresample)
        print(file)
        ds=xr.open_dataset(file)
        data[pt][Rep]=ds

In [ ]:
fig, ax = plt.subplots()
drag_meanlist = []
drag_sdlist = []
history_meanlist = []
history_sdlist = []
legend = []
norm = 2.4397537833333334e-05
omega_earth =  7.2921e-5
pt = 'inertial_Rep_constant'
for Rep,color in zip(Replist, Magmalist):


    Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
    Uslip = Uslip.flatten()
    Uslip = Uslip[~np.isnan(Uslip)]
    # Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
    factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Rep)
    factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
    history_force = factor_history * np.sqrt(data[pt][Rep].History_Basset_y**2 + data[pt][Rep].History_Basset_y**2 ).values
    history_force.flatten()
    history_force = history_force[~np.isnan(history_force)] # [mask]
    drag_force = factor_drag * Uslip
    bins_h, pdf_h = make_lognormal_PDF(history_force/norm,nbins=50,norm=True, vmin = 1e-5,vmax  = 1e2)
    bins_d, pdf_d = make_lognormal_PDF(drag_force/norm,nbins=50,norm=True ,vmin =  1e-5,vmax  = 1e2)
    drag_meanlist.append(np.mean(drag_force))
    drag_sdlist.append(np.std(drag_force))
    history_meanlist.append(np.mean(history_force))
    history_sdlist.append(np.std(history_force))
    legend.append('Re$_p^{\\mathrm{In}}$ = '+f'{Rep}')
    print(drag_force.mean())
    ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = color)
    ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = color, label='_nolegend_')
    ax.set_xscale('log')


pt = 'inertial_drag_Rep'
Rep = None
Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
Uslip = Uslip.flatten()
Uslip = Uslip[~np.isnan(Uslip)]
Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Repvalue)

factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
history_force = factor_history * np.sqrt(data[pt][Rep].History_Basset_y**2 + data[pt][Rep].History_Basset_y**2 ).values
history_force.flatten()
history_force = history_force[~np.isnan(history_force)] # [mask]
drag_force = factor_drag * Uslip
factor_cor = 2* omega_earth * np.sin(55 * np.pi /180 )/B
cor_force = factor_cor * Uslip 
bins_h, pdf_h = make_lognormal_PDF(history_force/norm,nbins=50,norm=True, vmin = 1e-5,vmax  =1e2)
bins_d, pdf_d = make_lognormal_PDF(drag_force/norm,nbins=50,norm=True ,vmin =  1e-5,vmax  = 1e2)
bins_c, pdf_c = make_lognormal_PDF(cor_force/norm,nbins=50,norm=True ,vmin =  1e-5,vmax  = 1e2)
drag_meanlist.append(np.mean(drag_force))
drag_sdlist.append(np.std(drag_force))
history_meanlist.append(np.mean(history_force))
history_sdlist.append(np.std(history_force))
ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = 'cornflowerblue')
ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = 'cornflowerblue',label='_nolegend_')
ax.plot(bins_c[1:], pdf_c[1:]*(bins_c[1:]-bins_c[:-1]), ':', color = 'cornflowerblue',label='_nolegend_')
legend.append('Flexible Re$_p$')
print(drag_force.mean())
ax.legend(legend,fontsize=18)#,loc=1)
ax.set_ylabel('PDF')
ax.set_xlabel('Force / $\\langle F_{\\mathrm{drag}}\\rangle$')
ax.text(4,0.04,'Drag',color='black',fontsize=18)
ax.text(1e-4,0.04,'History',color='black',fontsize=18)
# ax.set_xlim(1e-8,1e-2)
# ax.set_ylim(-0.001,0.07)
drag_meanlist = np.asarray(drag_meanlist)
drag_sdlist = np.asarray(drag_sdlist)
history_meanlist = np.asarray(history_meanlist)
history_sdlist = np.asarray(history_sdlist)

fig.tight_layout()
# fig.savefig('../figures/history_force/history_mei_drag_distributions.pdf')

In [ ]:
# history force for 95 persentile time
# density water
tau = 3196
av_temp_NWES = 11.983276
av_salinity_NWES = 34.70449
rho_water = 1027 # kg/m3 
dynamic_viscosity_water = dynamic_viscosity_Sharqawy(av_temp_NWES,av_salinity_NWES/1000)# 1.e-3# 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water
diameter = 0.25 # m
B=0.68



# srun --ntasks=1 --exclusive --nodes=1 python script_history_term_final_time.py --rep 0 --pt inertial_Rep_constant --twindow 0.1 --dt_resample 0.1  & 
# srun --ntasks=1 --exclusive --nodes=1 python script_history_term_final_time.py --rep 10 --pt inertial_Rep_constant --twindow 0.2 --dt_resample 0.2  & 
# srun --ntasks=1 --exclusive --nodes=1 python script_history_term_final_time.py --rep 100 --pt inertial_Rep_constant --twindow 1 --dt_resample 0.2  & 
# srun --ntasks=1 --exclusive --nodes=1 python script_history_term_final_time.py --rep 450 --pt inertial_Rep_constant --twindow 7 --dt_resample 0.2  & 
# srun --ntasks=1 --exclusive --nodes=1 python script_history_term_final_time.py --rep 1000 --pt inertial_Rep_constant --twindow 24.2 --dt_resample 0.2  & 
# srun --ntasks=1 --exclusive --nodes=1 python script_history_term_final_time.py --pt inertial_drag_Rep --twindow 2.8 --dt_resample 0.2  & 

Replist = [0,10,100,450,1000]
twindowlist = [0.1,0.2,1,7,24.2]
dt_resamplelist = [0.1,0.2,0.2,0.2,0.2]
path = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'
basefile_constant = '{pt}/history_term_Basset_Rep{Rep:04d}_twindow{twindow:05d}_tresample{tresample:04d}.netcdf'
basefile_flex ='{pt}/history_term_Basset_twindow{twindow:05d}_tresample{tresample:04d}.netcdf'
particle_types = ['inertial_Rep_constant','inertial_drag_Rep']

Repvalues = { 'inertial_Rep_constant':Replist,
             'inertial_drag_Rep':[None]
             
}
twindowvalues =  { 'inertial_Rep_constant':twindowlist,
             'inertial_drag_Rep':[2.8]
             
}
dt_resamplevalues = { 'inertial_Rep_constant':dt_resamplelist,
             'inertial_drag_Rep':[0.2]
             
}

basefiles = {'inertial_Rep_constant':path+basefile_constant, 
             'inertial_drag_Rep':path+basefile_flex}


In [ ]:
#settings
# twindow = 20 #h
tresample = 1#s

data = {}
for pt in particle_types:
    data[pt]={}

for pt in particle_types:
    for Rep,twindow,tresample in zip(Repvalues[pt],twindowvalues[pt],dt_resamplevalues[pt]):
        file = basefiles[pt].format(pt=pt,
                                    Rep=Rep,
                                    twindow=int(10*twindow),
                                    tresample=int(10*tresample))
        print(file)
        ds=xr.open_dataset(file)
        data[pt][Rep]=ds

In [ ]:
fig, ax = plt.subplots()
drag_meanlist = []
drag_sdlist = []
history_meanlist = []
history_sdlist = []
legend = []
norm = 2.4397537833333334e-05
omega_earth =  7.2921e-5
pt = 'inertial_Rep_constant'
for Rep,color in zip(Replist, Magmalist):


    Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
    Uslip = Uslip.flatten()
    Uslip = Uslip[~np.isnan(Uslip)]
    # Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
    factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Rep)
    factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
    history_force = factor_history * np.sqrt(data[pt][Rep].History_Basset_y**2 + data[pt][Rep].History_Basset_y**2 ).values
    history_force.flatten()
    history_force = history_force[~np.isnan(history_force)] # [mask]
    drag_force = factor_drag * Uslip
    bins_h, pdf_h = make_lognormal_PDF(history_force/norm,nbins=100,norm=True, vmin = 1e-5,vmax  = 1e2)
    bins_d, pdf_d = make_lognormal_PDF(drag_force/norm,nbins=100,norm=True ,vmin =  1e-5,vmax  = 1e2)
    drag_meanlist.append(np.mean(drag_force))
    drag_sdlist.append(np.std(drag_force))
    history_meanlist.append(np.mean(history_force))
    history_sdlist.append(np.std(history_force))
    legend.append('Re$_p^{\\mathrm{In}}$ = '+f'{Rep}')
    print(drag_force.mean())
    ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = color)
    ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = color, label='_nolegend_')
    ax.set_xscale('log')


pt = 'inertial_drag_Rep'
Rep = None
Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
Uslip = Uslip.flatten()
Uslip = Uslip[~np.isnan(Uslip)]
Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Repvalue)

factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
history_force = factor_history * np.sqrt(data[pt][Rep].History_Basset_y**2 + data[pt][Rep].History_Basset_y**2 ).values
history_force.flatten()
history_force = history_force[~np.isnan(history_force)] # [mask]
drag_force = factor_drag * Uslip
factor_cor = 2* omega_earth * np.sin(55 * np.pi /180 )/B
cor_force = factor_cor * Uslip 
bins_h, pdf_h = make_lognormal_PDF(history_force/norm,nbins=100,norm=True, vmin = 1e-5,vmax  =1e2)
bins_d, pdf_d = make_lognormal_PDF(drag_force/norm,nbins=100,norm=True ,vmin =  1e-5,vmax  = 1e2)
# bins_c, pdf_c = make_lognormal_PDF(cor_force/norm,nbins=25,norm=True ,vmin =  1e-5,vmax  = 1e2)
drag_meanlist.append(np.mean(drag_force))
drag_sdlist.append(np.std(drag_force))
history_meanlist.append(np.mean(history_force))
history_sdlist.append(np.std(history_force))
ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = 'cornflowerblue')
ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = 'cornflowerblue',label='_nolegend_')
# ax.plot(bins_c[1:], pdf_c[1:]*(bins_c[1:]-bins_c[:-1]), ':', color = 'cornflowerblue',label='_nolegend_')
legend.append('Flexible Re$_p$')
print(drag_force.mean())
ax.legend(legend,fontsize=18)#,loc=1)
ax.set_ylabel('PDF')
ax.set_xlabel('Force / $\\langle F_{\\mathrm{drag}}\\rangle$')
ax.text(4,0.04,'Drag',color='black',fontsize=18)
ax.text(1e-4,0.04,'History',color='black',fontsize=18)
# ax.set_xlim(1e-8,1e-2)
# ax.set_ylim(-0.001,0.07)
drag_meanlist = np.asarray(drag_meanlist)
drag_sdlist = np.asarray(drag_sdlist)
history_meanlist = np.asarray(history_meanlist)
history_sdlist = np.asarray(history_sdlist)

fig.tight_layout()
# fig.savefig('../figures/history_force/history_mei_drag_distributions.pdf')

In [ ]:
history_term_Basset_Rep0000_twindow00001_tresample00001.netcdf
history_term_Basset_Rep0000_twindow00001_tresample0001.netcdf

In [ ]:
fig, ax = plt.subplots()
# drag_meanlist = []
# drag_sdlist = []
# history_meanlist = []
# history_sdlist = []
legend = []
omega_earth =  7.2921e-5
norm = 2.4397537833333334e-05
pt = 'inertial_Rep_constant'
for Rep,color in zip(Replist, Magmalist):


    Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
    Uslip = Uslip.flatten()
    # select = ~np.isnan(Uslip)
    
    # Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
    factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Rep)
    factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
    history_force = factor_history * np.sqrt(data[pt][Rep].History_Basset_y**2 + data[pt][Rep].History_Basset_y**2 ).values
    history_force=history_force.flatten()
    cor_force = (2*omega_earth*np.sin(55*np.pi/180) /B )/ factor_drag
    ax.axvline(cor_force,color=color,zorder=-20,label='_nolegend_')
    drag_force = factor_drag * Uslip
    test = history_force/drag_force
    test = test[~np.isnan(test)]
    bins_h, pdf_h = make_lognormal_PDF(test,nbins=50,norm=True, vmin = 1e-5,vmax  = 1e2)
    # bins_d, pdf_d = make_lognormal_PDF(drag_force/norm,nbins=50,norm=True ,vmin =  1e-5,vmax  = 1e2)
    # drag_meanlist.append(np.mean(drag_force))
    # drag_sdlist.append(np.std(drag_force))
    # history_meanlist.append(np.mean(history_force))
    # history_sdlist.append(np.std(history_force))
    legend.append('Re$_p^{\\mathrm{In}}$ = '+f'{Rep}')
    # print(drag_force.mean())
    ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = color)
    # ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = color, label='_nolegend_')
    ax.set_xscale('log')


pt = 'inertial_drag_Rep'
Rep = None
Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2).values
Uslip = Uslip.flatten()
# select = ~np.isnan(Uslip)

Repvalue = Re_particle(Uslip,diameter=diameter, kinematic_viscosity=kinematic_viscosity_water)
factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Repvalue)
factor_drag_mean = factor_drag[~np.isnan(factor_drag)].mean()
factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
history_force = factor_history * np.sqrt(data[pt][Rep].History_Basset_y**2 + data[pt][Rep].History_Basset_y**2 ).values
history_force=history_force.flatten()
# history_force = history_force[select]#[~np.isnan(history_force)] # [mask]
drag_force = factor_drag * Uslip
cor_force = (2*omega_earth*np.sin(55*np.pi/180) /B )/ factor_drag_mean
test = history_force/drag_force
test = test[~np.isnan(test)]
# Uslip = Uslip[select]
bins_h, pdf_h = make_lognormal_PDF(test,nbins=50,norm=True, vmin = 1e-5,vmax  = 1e2)
# bins_h, pdf_h = make_lognormal_PDF(history_force/norm,nbins=50,norm=True, vmin = 1e-5,vmax  =1e2)
# bins_d, pdf_d = make_lognormal_PDF(drag_force/norm,nbins=50,norm=True ,vmin =  1e-5,vmax  = 1e2)
# drag_meanlist.append(np.mean(drag_force))
# drag_sdlist.append(np.std(drag_force))
# history_meanlist.append(np.mean(history_force))
# history_sdlist.append(np.std(history_force))
ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '--', color = 'cornflowerblue')
ax.axvline(cor_force,color='cornflowerblue',label='_nolegend_')
# ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '-', color = 'cornflowerblue',label='_nolegend_')
legend.append('Flexible Re$_p$')
# print(drag_force.mean())
ax.legend(legend,fontsize=18)#,loc=1)
ax.set_ylabel('PDF')
ax.set_xlabel('$F_{\\mathrm{history}}/F_{\\mathrm{drag}}$')
# ax.text(4,0.04,'Drag',color='black',fontsize=18)
# ax.text(1e-4,0.04,'History',color='black',fontsize=18)
# ax.set_xlim(1e-8,1e-2)
# ax.set_ylim(-0.001,0.07)
# drag_meanlist = np.asarray(drag_meanlist)
# drag_sdlist = np.asarray(drag_sdlist)
# history_meanlist = np.asarray(history_meanlist)
# history_sdlist = np.asarray(history_sdlist)


fig.tight_layout() 
# fig.savefig('../figures/history_force/history_mei_drag_distributions.pdf')

In [ ]:
np.array([2.0747591e-05,2.4186189e-05,2.6515958e-05,2.7168688e-05,2.7376494e-05,2.0390307e-05]).mean()

In [ ]:
fig, ax =plt.subplots()
# ax.errorbar(Replist,drag_meanlist,drag_sdlist,fmt='-o')
# ax.errorbar(Replist,history_meanlist,history_sdlist,fmt='--s')
ilist = [0,1,2,3,4]
for i, color in zip(ilist,Magmalist):
    # ax.plot(i,history_meanlist[i]/drag_meanlist[i],'s',color=color)
    # ax.errorbar(i,drag_meanlist[i],drag_sdlist[i]/2,fmt='o',color=color)
    # ax.errorbar(i,history_meanlist[i],history_sdlist[i]/2,fmt='s',color=color)
    ax.plot(i,history_meanlist[i]/drag_meanlist[i],'o',color=color)
# ax.plot(4,history_meanlist[4]/drag_meanlist[4],'s',color='cornflowerblue')
i=5
# ax.errorbar(i,drag_meanlist[i],drag_sdlist[i]/2,fmt='o',color='cornflowerblue')
# ax.errorbar(i,history_meanlist[i],history_sdlist[i]/2,fmt='s',color='cornflowerblue')
ax.plot(i,history_meanlist[i]/drag_meanlist[i],'s',color='cornflowerblue')
ax.set_xticks([0,1,2,3,4,5])
ax.set_xticklabels([0,10,100,450,1000,'flexible'])
# ax.set_yscale('log')
# drag_meanlist

In [ ]:
pt = 'inertial_Rep_constant'
Rep = 10
fig, ax = plt.subplots()
bin, pdf = make_PDF(data[pt][Rep].uslip.values,nbins=500,norm=True)
ax.plot(bin,pdf)
bin, pdf = make_PDF(data[pt][Rep].vslip.values,nbins=500,norm=True)
ax.plot(bin,pdf)

In [ ]:
pt = 'inertial_drag_Rep'
Rep = None
Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2)
Repvalues = Re_particle(Uslip,diameter=diameter,kinematic_viscosity=kinematic_viscosity_water)
factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Repvalues)
factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
history_force = factor_history *  np.sqrt(data[pt][Rep].History_Mei_x**2 + data[pt][Rep].History_Mei_y**2 )
drag_force = factor_drag * Uslip
bins_h, pdf_h = make_lognormal_PDF(history_force.values,nbins=100,norm=True, vmin = 1e-8,vmax  =1e-3)
bins_d, pdf_d = make_lognormal_PDF(drag_force.values,nbins=100,norm=True ,vmin =  1e-8,vmax  = 1e-3)
fig, ax = plt.subplots()
ax.plot(bins_h[1:], pdf_h[1:]*(bins_h[1:]-bins_h[:-1]), '-', color = 'navy')
ax.plot(bins_d[1:], pdf_d[1:]*(bins_d[1:]-bins_d[:-1]), '--', color = 'firebrick')
ax.set_xscale('log')


In [ ]:
mean_h = [] 
mean_d = [] 
sd_h = []
sd_d = []
pt = 'inertial_Rep_constant'
for Rep in Replist:
    print(Rep)
    Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2)
    factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Rep)
    factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
    history_force = factor_history * np.sqrt(data[pt][Rep].History_Mei_x**2 + data[pt][Rep].History_Mei_y**2 )
    history_force = history_force[~np.isnan(Uslip)]
    drag_force = factor_drag * Uslip[~np.isnan(Uslip)]
    mean_h.append(np.mean(history_force.values))
    mean_d.append(np.mean(drag_force.values))
    sd_h.append(np.std(history_force.values))
    sd_d.append(np.std(drag_force.values))

pt = 'inertial_drag_Rep'
Rep = None
Uslip = np.sqrt(data[pt][Rep].uslip**2 + data[pt][Rep].vslip**2)
Repvalue = Re_particle(Uslip,diameter=diameter,kinematic_viscosity=kinematic_viscosity_water)
factor_drag = prefactor_drag(B,kinematic_viscosity_water,diameter,Repvalue)
factor_history = prefactor_history(B,kinematic_viscosity_water,diameter)
history_force = factor_history *  np.sqrt(data[pt][Rep].History_Mei_x**2 + data[pt][Rep].History_Mei_y**2 )
history_force = history_force[~np.isnan(Uslip)]
drag_force = factor_drag[~np.isnan(Uslip)] * Uslip[~np.isnan(Uslip)]
mean_h.append(np.mean(history_force.values))
mean_d.append(np.mean(drag_force.values))
sd_h.append(np.std(history_force.values))
sd_d.append(np.std(drag_force.values))

In [ ]:
fig, ax = plt.subplots()
ax.plot([1,2,3,4,5],np.array(mean_h)/np.array(mean_d),'--o')
# ax.plot([1,2,3,4,5],, '--s')
# ax.set_yscale('log')

In [ ]:
print(prefactor_drag(B,kinematic_viscosity_water,diameter,1000))

In [ ]:
Uslip[~np.isnan(Uslip)].size

In [ ]:
np.sqrt(data['inertial_drag_Rep'][None].History_Mei_x**2 + data['inertial_drag_Rep'][None].History_Mei_y**2 )

angle